## Catatan :
- Dataset dari tweet english dari 3 negara (Singapura, Thailand, Vietnam) ditarik 1 persatu kemudian di merge
- menggunakan API Microsoft Text Analytic
- Preprocess : 
  - menghapus duplicat content
  - menghapus link dan url image twitter
  - menghapus row 2818 <--- ini di cek manual aja di text analytics
  - dieksport per 1000 karna api cuma bisa handle 1000

In [38]:
import urllib
import json
from http.client import HTTPSConnection
import pandas as pd
import csv
import re

In [65]:
dataset=pd.read_csv("gojekallcountry.csv", encoding="ISO-8859–1")
dataset = dataset[['id', 'Country','tweet']] 

In [66]:
print(len(dataset))
dataset = dataset.drop_duplicates('id', keep='last')
print(len(dataset))

3011
2849


In [67]:
dataset.to_csv('gojekallcountry-no-duplicate.csv', index=False) 

In [111]:
docs=[]
with open('gojekallcountry-no-duplicate-3.csv','r',encoding='utf-8') as csvFile:
        csvReader = csv.reader(csvFile, delimiter=',')
        next(csvReader, None)
        for row in csvReader:
            tweetnew = re.sub(r"http\S+", "", row[2])
            tweetnew = re.sub(r"pic.twitter.com\S+", "",tweetnew)
            docs.append({"id":row[0] ,"text":tweetnew})

In [112]:
uri = 'gojeksentiment.cognitiveservices.azure.com'
accessKey = 'df79dfd31c0b434d9c2274eb90089372--'

def GetSentiment (documents):
    path = '/text/analytics/v2.0/sentiment'
    
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = HTTPSConnection (uri)
    body = json.dumps (documents)
    conn.request ("POST", path, body, headers)
    response = conn.getresponse ()
    return response.read ()

documents = {
    'documents': docs
}

In [113]:
result = GetSentiment(documents)

In [114]:
sentiments = json.loads(result)["documents"]

In [115]:
link = pd.DataFrame(columns=['id','score'])
for x in sentiments:
    link = link.append({'id': x["id"],'score': x["score"]}, ignore_index=True)

In [116]:
link

,id,score
0,2000,0.500000
1,2001,0.500000
2,2002,0.500000
3,2003,0.500000
4,2004,0.500000
5,2005,0.500000
6,2006,0.500000
7,2007,0.500000
8,2008,0.247408
9,2009,0.500000


In [117]:
link.to_csv('result-3.csv', index=False) 